In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# place all *.py files in the same folder
import lib2
from lib2 import Predicate, optimize
from models import customXGB
from apriori import preprocessDataset, runApriori, aprioriout2predicateList

import matplotlib.pyplot as plt

In [2]:
DATAFILE = '../adult.data' # location of dataset
random_state = None # change to something for exactly reproducible results
sensitive_attribute = "Sex"
target_name = "label"
positive_label = ">50K"
negative_label = "<=50K"
model_train_fraction = 0.7

In [3]:
# specify feature names
feature_names = [
   "Age", "Workclass", "fnlwgt", "Education",
   "Education-Num", "Marital Status", "Occupation",
   "Relationship", "Race", "Sex", "Capital Gain",
   "Capital Loss", "Hours per week", "Country", "label"
]
# specify categorical columns
cate_columns = ['Workclass', 'Education', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']

In [4]:
# define featureCost as a mapping holding, for each feature name, the respective cost of not keeping that feature constant
# Any feature not specified will have its cost set to 1
featureCosts = {"Sex": 100}

# define featureChange as a mapping from str to function, holding, for each feature name, the respective 
# function that calculates the cost of change from one value to another.
# Any feature change not specified will be set to 1 if there is change, and 0 otherwise.
def age_cost(age1: str, age2: str) -> int:
    return abs(int(age1) - int(age2))
featureChange = {"Age": age_cost}

# set the weights that manage the relative influence of coverage, correctness, feature cost and feature change
# in the objective function of the algorithm
l_cover = 1
l_correct = 1
l_cost = 1
l_change = 1

In [5]:
lib2.setFeatureCost(featureCosts)
lib2.setFeatureChange(featureChange)
lib2.set_lambdas(l_cover, l_correct, l_cost, l_change)

# Data loading

Loads the dataset into variable `data`. Just run it.

In [6]:
data = pd.DataFrame(
  np.genfromtxt(DATAFILE, delimiter=', ', dtype=str),
  columns=feature_names
)

# Train test split

Split into train-test. The train set is used specifically for training the model, and nothing further.

In [7]:
X = data.drop(target_name, axis=1)
y = data[target_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=model_train_fraction, random_state=random_state)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Definition and Training of a black-box model

We use a black box model based on gradient boosted decision trees.

In [8]:
model = customXGB(n_estimators=300, max_depth=5)
model.fit(X_train, y_train, cate_columns=cate_columns)

0 denotes the negative and 1 the positive class.

In [9]:
model.predict(X_test.iloc[:100, :])

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Testing the model

We check if the model is satisfactory.

In [10]:
preds = model.predict(X_test)
print(classification_report(y_test.map({negative_label: 0, positive_label: 1}), preds))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      7407
           1       0.75      0.65      0.70      2362

    accuracy                           0.86      9769
   macro avg       0.82      0.79      0.80      9769
weighted avg       0.86      0.86      0.86      9769



# Finding the affected

We check the predictions of the model on the test data, i.e. our population. The ones that get a 0 (negative outcome) are called the affected individuals.

In [11]:
X_aff_idxs = np.where(model.predict(X_test) == 0)[0]
print(f"Test data shape: {X_test.shape}")
X_aff = X_test.iloc[X_aff_idxs, :]
print(f"Affected shape: {X_aff.shape}")
# X_aff.reset_index(inplace=True, drop=True)

Test data shape: (9769, 14)
Affected shape: (7706, 14)


This means that the model classifies into the negative class about 7000 out of the 10000 total (test set) individuals.

# Running apriori adaptation

We now generate the frequent itemsets of the datasets. These are used by the global counterfactual generating algorithm, in order to try and cover as many affected individuals as possible.

Here, we have implemented a simple `runApriori` wrapper function, which basically runs the fpgrowth algorithm for frequent itemset mining of the library mlxtend. It returns a dataframe with two columns, an $itemset$ column which contains the itemsets, in the form value tuples, and a $support$ column, which contains the relative frequency with which the itemset is contained in the dataset. Meaning, the fraction of individuals which have this specific combination of feature values.

Notice that we can give a minimum support as an argument to the function. This means that it returns only those itemsets whose support are above this value.

*Note*: You can ignore the "+feature_name" part. It has been appended to every value for implementation reasons, so that we know the "type" of each value, i.e. the feature it corresponds to. For example, whether a 0 is a value for "Capital Loss" or for "Capital Gain".

In [12]:
d = X_test.drop([sensitive_attribute], axis=1)
freq_itemsets = runApriori(preprocessDataset(d), min_support=0.03)
freq_itemsets.reset_index()
print(freq_itemsets.head())

     support                          itemsets
0   0.951377                  (0+Capital Loss)
1   0.917699                  (0+Capital Gain)
2   0.898147           (United-States+Country)
55  0.869076  (0+Capital Loss, 0+Capital Gain)
8   0.860477                      (White+Race)


In [13]:
print(freq_itemsets.head(100).to_string())

       support                                                                                                      itemsets
0     0.951377                                                                                              (0+Capital Loss)
1     0.917699                                                                                              (0+Capital Gain)
2     0.898147                                                                                       (United-States+Country)
55    0.869076                                                                              (0+Capital Loss, 0+Capital Gain)
8     0.860477                                                                                                  (White+Race)
56    0.854130                                                                       (0+Capital Loss, United-States+Country)
57    0.822193                                                                       (0+Capital Gain, United-States+Country)


In [14]:
print(freq_itemsets.tail(100))
# print(freq_itemsets.tail(100).to_string()) # uncomment for more details

       support                                           itemsets
1368  0.030402  (HS-grad+Education, 0+Capital Gain, 9+Educatio...
1365  0.030402  (0+Capital Loss, 9+Education-Num, Never-marrie...
1367  0.030402  (0+Capital Loss, HS-grad+Education, 9+Educatio...
4394  0.030402  (12+Education-Num, Assoc-acdm+Education, White...
4390  0.030402                     (12+Education-Num, White+Race)
...        ...                                                ...
3510  0.030095  (Husband+Relationship, 40+Hours per week, 0+Ca...
4095  0.030095  (0+Capital Loss, United-States+Country, Adm-cl...
4061  0.030095  (Married-civ-spouse+Marital Status, Adm-cleric...
790   0.030095  (40+Hours per week, Black+Race, United-States+...
5688  0.030095  (13+Education-Num, Exec-managerial+Occupation,...

[100 rows x 2 columns]


Next, we use the function `aprioriout2predicateList`, which "casts" the output of the frequent itemset mining algorithm to our internal representation of a "triple" (as in the ares paper). This representation is the class `Predicate`.

RL is the initial set of candidate predicates (taken as the output of the itemset algo), from which we will then pick pairs to represent our rules (as in ares paper).

In [15]:
RL = aprioriout2predicateList(freq_itemsets)
pprint(RL[:10])
print(len(RL))

[Predicate(features=['Capital Loss'], values=['0']),
 Predicate(features=['Capital Gain'], values=['0']),
 Predicate(features=['Country'], values=['United-States']),
 Predicate(features=['Capital Loss', 'Capital Gain'], values=['0', '0']),
 Predicate(features=['Race'], values=['White']),
 Predicate(features=['Capital Loss', 'Country'], values=['0', 'United-States']),
 Predicate(features=['Capital Gain', 'Country'], values=['0', 'United-States']),
 Predicate(features=['Capital Loss', 'Race'], values=['0', 'White']),
 Predicate(features=['Country', 'Race'], values=['United-States', 'White']),
 Predicate(features=['Capital Gain', 'Race'], values=['0', 'White'])]
5771


# Running the optimization procedure

First, just turn the user-defined SD (subgroup descriptors) to predicates.

In [16]:
SD = list(map(Predicate.from_dict, [
    {sensitive_attribute: val} for val in data[sensitive_attribute].unique()
]))
print(SD)

[Predicate(features=['Sex'], values=['Male']), Predicate(features=['Sex'], values=['Female'])]


In [17]:
rules = [(Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Capital Gain', 'Relationship'], values=['Bachelors', '0', 'Husband']),
   Predicate(features=['Education', 'Capital Gain', 'Relationship'], values=['Some-college', '0', 'Husband'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Workclass', 'Marital Status', 'Relationship'], values=['Private', 'Divorced', 'Not-in-family']),
   Predicate(features=['Workclass', 'Marital Status', 'Relationship'], values=['Private', 'Married-civ-spouse', 'Wife'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['9']),
   Predicate(features=['Education-Num'], values=['14'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['10']),
   Predicate(features=['Education-Num'], values=['14'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Marital Status'], values=['Bachelors', 'Never-married']),
   Predicate(features=['Education', 'Marital Status'], values=['Some-college', 'Married-civ-spouse'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status', 'Education-Num'], values=['Married-civ-spouse', '10']),
   Predicate(features=['Marital Status', 'Education-Num'], values=['Married-civ-spouse', '13'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['11']),
   Predicate(features=['Education-Num'], values=['14'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['7']),
   Predicate(features=['Education-Num'], values=['14'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status', 'Education'], values=['Married-civ-spouse', 'Bachelors']),
   Predicate(features=['Marital Status', 'Education'], values=['Married-civ-spouse', 'HS-grad'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status', 'Relationship'], values=['Divorced', 'Unmarried']),
   Predicate(features=['Marital Status', 'Relationship'], values=['Married-civ-spouse', 'Wife'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Capital Loss', 'Race'], values=['Masters', '0', 'White']),
   Predicate(features=['Education', 'Capital Loss', 'Race'], values=['Some-college', '0', 'White'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Education-Num'], values=['Assoc-acdm', '12']),
   Predicate(features=['Education', 'Education-Num'], values=['Masters', '14'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Capital Loss', 'Education-Num', 'Capital Gain', 'Marital Status'], values=['0', '9', '0', 'Married-civ-spouse']),
   Predicate(features=['Capital Loss', 'Education-Num', 'Capital Gain', 'Marital Status'], values=['0', '13', '0', 'Married-civ-spouse']))]

In [18]:
from recourse_sets import TwoLevelRecourseSet

In [19]:
rules = TwoLevelRecourseSet.from_triples(rules)

In [20]:
model.predict(X_test)

array([0, 1, 0, ..., 0, 1, 0])

In [21]:
from metrics import incorrectRecoursesSubmodular, incorrectRecourses, cover

In [22]:
%%time
print(incorrectRecoursesSubmodular(rules, X_aff, model))

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
4839
CPU times: total: 16 s
Wall time: 1.97 s


In [23]:
%%time
print(incorrectRecourses(rules, X_aff, model, percentage=False))

4839
CPU times: total: 3.95 s
Wall time: 3.51 s


In [25]:
%%time
print(cover(rules, X_aff))

6194
CPU times: total: 1.17 s
Wall time: 585 ms


Now, we run the submodular optimization.

In [26]:
%%time

final_rules = optimize(SD, RL, X_aff, model)

Total triples = 12836
X_aff shape before: (7754, 14)
Calculated incorrect recourse for each triple
Calculated feature costs for each triple
Calculated feature changes for each feature
X_aff shape after: (7754, 14)
Calculated covers for each triple
141
CPU times: total: 3h 57min 21s
Wall time: 46min 35s


In [27]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Capital Gain', 'Relationship'], values=['Bachelors', '0', 'Husband']),
   Predicate(features=['Education', 'Capital Gain', 'Relationship'], values=['Some-college', '0', 'Husband'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Workclass', 'Marital Status', 'Relationship'], values=['Private', 'Divorced', 'Not-in-family']),
   Predicate(features=['Workclass', 'Marital Status', 'Relationship'], values=['Private', 'Married-civ-spouse', 'Wife'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['9']),
   Predicate(features=['Education-Num'], values=['14'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['10']),
   Predicate(features=['Education-Num'], values=['14'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Marital Status'], value

In [20]:
print(X_aff.shape)

(7754, 14)


In [19]:
%%time

final_rules = optimize(SD, RL[:400], X_aff[:400], model)

Total triples = 364
X_aff shape before: (400, 14)
Calculated incorrect recourse for each triple
Calculated feature costs for each triple
Calculated feature changes for each feature
X_aff shape after: (400, 14)
Calculated covers for each triple
24
CPU times: total: 50.1 s
Wall time: 6.28 s


In [20]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['10']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education', 'Education-Num'], values=['HS-grad', '9']),
   Predicate(features=['Education', 'Education-Num'], values=['Bachelors', '13']))],
 -176,
 222,
 -4,
 -4)


In [19]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Relationship'], values=['Husband']),
   Predicate(features=['Relationship'], values=['Not-in-family'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse']))],
 -5121,
 5464,
 -3,
 -3)


In [30]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['10']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['9']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education'], values=['Bachelors']),
   Predicate(features=['Education'], values=['Some-college']))],
 -4085,
 4845,
 -4,
 -4)


In [30]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['10']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['9']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education'], values=['Bachelors']),
   Predicate(features=['Education'], values=['Some-college']))],
 -4085,
 4845,
 -4,
 -4)


In [19]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse']))],
 -1248,
 1362,
 -1,
 -1)


In [38]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse']))],
 -67,
 72,
 -1,
 -1)


In [19]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Marital Status'], values=['Never-married']),
   Predicate(features=['Marital Status'], values=['Married-civ-spouse']))],
 -70,
 75,
 -1,
 -1)


In [29]:
pprint(final_rules)

([(Predicate(features=['Sex'], values=['Female']),
   Predicate(features=['Education-Num'], values=['9']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['9']),
   Predicate(features=['Education-Num'], values=['13'])),
  (Predicate(features=['Sex'], values=['Male']),
   Predicate(features=['Education-Num'], values=['10']),
   Predicate(features=['Education-Num'], values=['13']))],
 -159,
 206,
 -3,
 -3)
